In [1]:
%%capture
# !pip install -q -U openai langchain-openai anthropic docling easyocr tenacity chime

### PARSE PDF

In [15]:
# import logging
# import time
# from pathlib import Path
# from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
# from docling.datamodel.base_models import FigureElement, InputFormat, Table
# from docling.datamodel.pipeline_options import PdfPipelineOptions
# from docling.document_converter import DocumentConverter, PdfFormatOption
# _log = logging.getLogger(__name__)
# IMAGE_RESOLUTION_SCALE = 5.0
# logging.basicConfig(level=logging.INFO)

# input_doc_path = Path("/kaggle/input/plumbing/Wild Pine Plumbing Service Call Tree.pdf")
# output_dir = Path("imgs")

# pipeline_options = PdfPipelineOptions()
# pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
# pipeline_options.generate_page_images = True
# pipeline_options.generate_picture_images = True

# doc_converter = DocumentConverter(
#     format_options={
#         InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
#     }
# )

# start_time = time.time()

# conv_res = doc_converter.convert(input_doc_path)

# output_dir.mkdir(parents=True, exist_ok=True)
# doc_filename = conv_res.input.file.stem

# # Save page images
# for page_no, page in conv_res.document.pages.items():
#     page_no = page.page_no
#     page_image_filename = output_dir / f"{doc_filename}-{page_no}.jpeg"
#     with page_image_filename.open("wb") as fp:
#         page.image.pil_image.save(fp, format="JPEG")

# # Save images of figures and tables
# table_counter = 0
# picture_counter = 0
# for element, _level in conv_res.document.iterate_items():
#     if isinstance(element, TableItem):
#         table_counter += 1
#         element_image_filename = (
#             output_dir / f"{doc_filename}-table-{table_counter}.jpeg"
#         )
#         with element_image_filename.open("wb") as fp:
#             element.get_image(conv_res.document).save(fp, "JPEG")

#     if isinstance(element, PictureItem):
#         picture_counter += 1
#         element_image_filename = (
#             output_dir / f"{doc_filename}-picture-{picture_counter}.jpeg"
#         )
#         with element_image_filename.open("wb") as fp:
#             element.get_image(conv_res.document).save(fp, "JPEG")

# end_time = time.time() - start_time

# _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")

## GET NODES FOR PROMPT

In [16]:
at_nodes = ['AT Plumbing Service Service Call Tree',
 'Call Arrival',
 'AT Plumbing cenvice How may (helpyou today?',
 'Customer Service (Past customer or customer with current booking)',
 'Lead',
 'Personal Call',
 'Salesperson',
 '(needs plumbing services)',
 'Calling about UPCOMING booking',
 'Calling about a PASTICOMPLETED booking',
 'Collect and log Basic Information in Talkdesk Contact: Name Address Phone Number Email Address',
 'Tellthe caller you Vill pass along a messuge; collect: Name Callbad Number, Message',
 'Politely decling and insist you te Ooxnmeresed',
 'Determine what the customer needs help with',
 'Determine the rEason for thelr',
 'It is urgent / customer is upset',
 'Customer wants t0 book another appointment',
 'Record Name, callback number; issue and reach out t0 Andrew with the issue via SMS at any time of the day',
 'Verify customer iS in service range using customer Map',
 'Customer wants tO change an upcoming appointment',
 'Customer wants an updated arrival window',
 'Customer wants t0 cancel upcoming appointment',
 'Customer has a non-urgent question about their service',
 'IN service range',
 'Duringlafter the quoted window',
 'Before the quoted window',
 'NOT in service range',
 'Appointment is TODAY',
 'Appointment NOT today',
 'Greatl Now hat seems {0 bc the problem?',
 'Change the "ppointment Jrd send an SMS message t0 the Ownat Immedlately t0 inform them ot Ehe €ame scheduling changes',
 'Tell cusromer you will try to conacina plumber Cal Andrew asking about his Jitival Follow up with customer J> needed',
 'ntorm the cuetocner that their current Jrrival window Ts ~Stll the most accurate InfarmationThe plumbar wll contact when he Is on the way',
 '1) Cancel the Jppoinunent 2) Ask customet why they are cancelling and record In call noies 31 i same day; send SMS t0 Andrer',
 'TRecordthe customer , nume contact Info;and question and pue into the Summary On Service Iitan',
 'Complete Jny requlred folow- VD Kth the customer',
 'Refer t0 Knowledge Base t0 capture relevant information and verify we can service.',
 'DOES NOT provide this service Gaswater heaters Mobile homes Sewver blockages or backups BackOlcw Issues Multiple clogged sinks; drains tollets ~Pipe bursting Suni flov Or up-flush toilet installations Nerv boiler sysrems',
 'Confirm customers distance from Portland, ME using Service Zone Lookup tool',
 'Customer is more than 60 minutes away',
 'Needs ESTIMATE booking-',
 'Needs SERVICE CALL booking',
 'Determine if the service is an emergency Is your house currently flooding? Can you shut the water offido you know where the shutoffs are? How long has this been going on for?',
 'Determine how far the customer is',
 '~It is NOT an emergency',
 'It is an emergency (needs service ASAP',
 'More than 45 mins_',
 '45-60 min;',
 'Disclose $90 dispatch (ee CREDITABLE t0 project cost Schedule next Jvailhe Eslmae appo niment',
 'Disclose $139.50 ~dispatch (ee Scnedulenext Avalable @Shmaie oppointment',
 'Regular appointment- booking',
 'Disclose $285 emergency dispatch fee',
 'During business hours',
 'After Hours',
 'Schedule',
 'Schedule Mcl avallable Contac Andrev [O S2fhe can senvicaine emecgency cooner Follova UD vath customer I needud',
 "aopointment for Mst appolntment the next morning Call Andreva if Inue emergency ate) Commic (0 'folowing Up with GuSOnter",
 'Log cunstomet Anfocmaticnin Talkdesk and let the customer know we are undble t0 sence Enem',
 'Disclose $139 Dispatch Fee',
 'Bock customer for next aallable uppointrent in ServiceTitan']
wildpine_nodes = ['Wild Pine Plumbing Service Call Tree',
 'Call Arrival',
 'Wild Pine',
 'Plumbing ! How may | help you today?',
 'Customer Service (Past customer r customer with current booking)',
 'Lead',
 'Personal Call',
 'Salesperson',
 '(needs plumbing services)',
 'Caling about UPCOMING booking',
 'Caling about a PASTICOMPLETED booking',
 'Collect and log Basic Information in Talkdesk Contact: Name Address Phone Number Email Address',
 'Tell the caller you will pass along message, collect: Name, Callback Number; Message',
 'Take a message and share with Brandon via SMS (except from Yelp, do not take message)',
 'Determine what the customer needs help with',
 'Determine the reason for their call',
 'It is urgent / customer is upset',
 'Customer wants to book another appointment',
 'Record Name, callback number; issue and reach out to Brandon with the issue via SMS at any time of the day',
 'Verify customer is in service range using customer Map',
 'Customer wants to change an upcoming appointment',
 'Customer wants an updated arrival window',
 'Customer wants to cancel upcoming appointment',
 'Customer has a non-urgent question about their service',
 'IN service range',
 'During/after the quoted window',
 'Before the',
 'Appointment is TODAY Appointment NOT today',
 'quoted window',
 'Change the appointment and send an SMS message to the owner immediately to inform them of the same day scheduling changes',
 'Tel customer you will try to contact the plumber Send SMS to Brandon asking about his arrival Follow up with customer as needed',
 'Inform the customer that their current arrival window is still - the most accurate information. The plumber will contact when he is on the way',
 '1) Cancel the appointment 2) Ask customer why they are cancelling and record in call notes 3) If same day; send SMS to Brandon',
 'NOT in service range',
 'Greatl Now what seems to be the problem?',
 "1) Record the customer's name; contact info, and question 2) Send non- urgent message to Brandon via SMS",
 'Complete any required follow- up with the customer',
 'Refer to Knowledge Base to capture relevant information and verify we can service.',
 'DOES NOT provide this service',
 'CAN provide service',
 'Determine if the service is an emergency Is your house currently flooding? Can you shut the water offIdo you know where the shutoffs are? How long has this been going on for?',
 'It is NOT an emergency',
 'It is an erergency (needs service ASAP)',
 'Regular appointment booking',
 'Disclose $400 emergency dispatch fee',
 'Availability for emergency',
 'No appts avail',
 'Contact Brandon to see if he can service the emergency',
 'Schedule',
 "Log ' customer information in Talkdesk and let the customer know we are unable to service them",
 'Disclose $89 Dispatch Fee',
 'appointment for next available that day or the first appointment the next morning',
 'Book customer for next available appointment in Housecall',
 'Follow up with customer, if needed']

redhead_nodes = ['RedHead Rooter Service Call Tree',
 'Call Arrival',
 'RedHead Rooterl How may | help you today?',
 'Immediately call the on-call technician and provide the customer contact information and address to them_ Follow up as needed:',
 'Customer Service (Past customer r customer with current booking)',
 'Lead',
 'Personal Call',
 'Salesperson',
 '(needs plumbing services)',
 'Angi Call',
 'Caling about UPCOMING booking',
 'Caling about a PASTICOMPLETED booking',
 'Collect and log Basic Information in Talkdesk Contact: Name Address Phone Number Email Address How they heard of RedHead Rooter',
 'Does not get personal calls on his business phone',
 "Politely decline and insist you're not interested",
 'Determine the reason for their call',
 'Determine what the customer needs help with',
 'It is urgent / customer is upset',
 'Customer wants to book another appointment',
 'Record the issue and reach out to Shane immediately even after hours (be sure to include indicators of urgency)',
 'Verify customer is in service range using customer Map',
 'Customer wants to change an upcoming appointment',
 'Customer wants an updated arrival window',
 'Customer wants to cancel upcoming appointment',
 'Customer has a non-urgent question about their service',
 'IN service range',
 'During/after the quoted window',
 'Before the',
 'Appointment is TODAY Appointment NOT today',
 'quoted window',
 '1) Apologize and say "Iam sorry the technician is running behind schedule_ They will contact you when they are on the way 2) send an SMS to tech to follow up with next CX',
 'Inform the customer that their current arrival window is still - the most accurate information: The tech will contact them when he is on the way',
 'Change the appointment and send an SMS the technician immediately to inform them of the same day scheduling changes',
 '1) Cancel the appointment 2) Ask customer why they are canceling and record in call notes 3) If same day; contact company',
 "1) Record the customer's name; contact info, and question 2) Send an SMS to the tech who did the original service",
 'NOT in service range',
 'Greatl Now what seems to be the problem?',
 'Complete any required follow- up with the customer',
 'Refer to Knowledge Base to capture relevant information and verify we can service.',
 'DOES NOT provide this service Septic pumping (will do other septic)',
 'CAN provide service',
 'Determine if the service is an emergency',
 'It is NOT an emergency',
 '#Ifyou are unsure, cal the on-cal tech to determine if it is an emegency',
 'It is an emergency (heeds service ASAP)',
 'Provide set fee pricing for drain cleaning service S125',
 'After hours (disclose there are free estimates; but there is an increased price for after hours appointments)',
 'Regular ppointment booking',
 'Basic drain cleanings',
 'During business hours',
 'Always say this comes with a free drain camera inspection check up to ensure there are no issues inside the drain',
 'Stil wants emergency appointment',
 'Open Housecall Pro and book customer for next available appointment time that customer is also available (If same day - inform owner via call)',
 'Call the on-call tech (schedule found in KB) to confirm availability and timing; follow up with the customer as needed',
 "Log ' customer information in Talkdesk and let the customer know we are unable to service them",
 'We are not currently providing call answering during business hours but would call Shane',
 'Call the',
 'technician the job is scheduled for immediately after booking to let them know there is a new job scheduled for them today',
 'Same Day Appt:']

readhead_img = "/kaggle/working/imgs/RedHead Rooter Service Call Tree-1.jpeg"
wildpine_img = "/kaggle/working/imgs/Wild Pine Plumbing Service Call Tree-1.jpeg"
at_img = "/kaggle/working/imgs/AT Plumbing Service Call Tree-1.jpeg"

choices = {
    "at": (at_img, at_nodes),
    "wildpine": (wildpine_img, wildpine_nodes),
    "redhead": (readhead_img, redhead_nodes),
}

main_diagram = "at"
if main_diagram in choices.keys():
    img_path = choices[main_diagram][0]
    nodes = choices[main_diagram][1]
print("Will test with", img_path)

Will test with /kaggle/working/imgs/AT Plumbing Service Call Tree-1.jpeg


In [17]:
# import easyocr
# reader = easyocr.Reader(['en'], )
# nodes = reader.readtext(img_path,
#                         detail = 0, paragraph=True,
#                         canvas_size=5000,
#                         decoder="wordbeamsearch",
#                         ## experiment below. Ref: https://www.jaided.ai/easyocr/documentation/
#                         beamWidth = 20,
#                         width_ths=0.9, 
#                         # height_ths=1.2,
#                         text_threshold=0.6,
#                         # x_ths=1,
#                         # y_ths=1.9,
#                         # ycenter_ths=0.7,
#                         mag_ratio=5.5,
#                         contrast_ths=0.1, 
#                         adjust_contrast=0.3,
#                         )
# nodes

### LINEARIZE DIAGRAM: ZERO-SHOT

In [18]:
# import os
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# anthropic_key = user_secrets.get_secret("ANTHROPIC_API_KEY")
# os.environ["ANTHROPIC_API_KEY"] = anthropic_key
# import anthropic
# import base64
# from textwrap import dedent

# def encode_image(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode("utf-8")


# image_path = img_path
# base64_image = encode_image(image_path)

# client = anthropic.Anthropic()

# prompt = """The diagram in the provided image is a workflow process that describes steps or nodes that an customer service human agent handles phone calls from a person in different cases.
# Your task is to interpret the diagram and generate workflow plan in YAML format.
# Each box means a step.
# ### Strictly comply with the following rules:
# - In the output, include all boxes and texts found in the diagram, and keep the texts as they are. Expand texts are they are in the original diagram.
# - In the output, each Step corresponds with a box of any shape (rectangle, haxagon or diamond). Include all the nodes until you find no more.
# - For a current decision step that leads to multiple possible steps, use if-clauses in the current step to show steps that may conditionally come next. This is crucial for your success.
# - In the If-clause or in every Step, always use "go to step_<step_id>" as the reference to the next step. This is extremely important.
# - Keep the original texts you found in the diagram intact. Don't change them.
# These are extremely important requirements you must follow.

# The following elements are steps, conditions and branches in the diagram. You have to consider ALL OF them together with the diagram when creating the output. Ensure to use all the elements in your output because they're the essential nodes and conditions.
# ### Elements to consider:
# {nodes}
# Go over each the elements and check you must include all the above elements in your output. 

# Here is an example illustrating the output template:
# step_0: |
#   - Determine user request intent
#     - If user request is about personal (not related to a plumbing request), go to step_1
#     - If user request is about selling something, go to step_2
#     - If user request is about a new lead (needs plumbing services), go to step_3
#     - If user request is about customer service (Past customer or customer with current booking), go to step_16

# step_1: |
#   - Tell the caller you will pass along a message and collect: Name Callback Number Message
#   - Follow up with customer, if needed

# step_2: |
#   - Politely decline and insist you're not interested
#   - End the call

# step_3: |
#   - Collect and log Basic Information in Talkdesk Contact: Name, Address, Phone Number, Email Address
#   - Then go to step_4

# step_4: |
#   - Verify customer is in service range using the customer Map
#     - If customer is NOT in service range, go to step_5
#     - If customer is in service range, go to step_6

# step_5: |
#   - Log customer information in Talkdesk Contact
#   - Let the customer know we are unable to service them
#   - End the call

# step_6: |
#   - Refer to Knowledge Base to capture relevant information and verify we can service
#     - If we cannot service, go to step_5
#     - Else, go to step_7
# """.format(nodes=nodes)
# prompt = """The following list includes all elements present in the diagram in the provided image. The name of the elements may be misspelt or cut off.
# Your task is to go over the list and give correct text for each element. Give a new corrected list.
# <list>
# {nodes}
# </list>""".format(nodes=nodes)
# message = client.messages.create(
#     model="claude-3-5-sonnet-20241022",
#     temperature=0.1,
#     max_tokens=4024,
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "image",
#                     "source": {
#                         "type": "base64",
#                         "media_type": "image/jpeg",
#                         "data": base64_image,
#                     },
#                 },
#                 {
#                     "type": "text",
#                     "text": prompt
#                 }
#             ],
#         }
#     ],
# )

# diagram_text = message.content[0].text
# print(diagram_text)

In [19]:
# reflect_prompt = dedent("""
# The following text is a way to describe the nodes and their relationships in the diagram from the provided image.
# <text>
# {diagram_text}
# </text>

# Here are a list of elements that an ideal text for the diagram should include:
# <elements>
# {elements}
# </elements>

# Your task to is meticulously comparing the text against each element of the element list to see if the text misses any elements. If it does, add those elements to the text, using the diagram as a reference.
# You should return:
# - All elements that are in the list but are not mentioned in the text. Go over the list elements one by one. Pay attention to branches and conditions of a step. 
# - A complete revised diagram text with these missing elements seamlessly incorporated into the appropriate sections. 
# The new text must be complete and contain no notes so that when you read it, you won't know that modifications are there. Add title "### Revised text" before the text.
# """).strip().format(diagram_text=diagram_text, elements=nodes)
# reflect_message = client.messages.create(
#     model="claude-3-5-sonnet-20241022",
#     temperature=0.3,
#     max_tokens=4024,
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "image",
#                     "source": {
#                         "type": "base64",
#                         "media_type": "image/jpeg",
#                         "data": base64_image,
#                     },
#                 },
#                 {
#                     "type": "text",
#                     "text": reflect_prompt
#                 }
#             ],
#         }
#     ],
# )

# reflect = reflect_message.content[0].text
# print(reflect)

In [20]:
# def extract_result(text: str) -> str:
#     if "### Revised text" in text:
#         revised_text = (text.split("### Revised text", 1)[1]
#                         .replace("```yaml", "")
#                         .replace("```", "")
#                         .strip()
#                        )
#     else:
#         revised_text = None 
#     return revised_text
# print(extract_result(reflect))

## LINEARIZE DIAGRAM: FEW-SHOT

In [21]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
anthropic_key = user_secrets.get_secret("ANTHROPIC_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = anthropic_key
import anthropic
import base64
from textwrap import dedent

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def correct_nodes(img_path: str, nodes) -> str:
    print("Correcting nodes...")
    image_path = img_path
    base64_image = encode_image(image_path)
    
    client = anthropic.Anthropic()
    items_to_remove = """Great! Now what seems to be the problem?"""
    prompt = """The following list includes all elements present in the diagram in the provided image. The name of the elements may be misspelt or cut off.
    Your task is to go over the list and give correct text for each element. Number the elements.
    In the new list, don't include: `{items_to_remove}`
    Give a new corrected list in this format: ## Corrected list: 

    Here's the original list
    <list>
    {nodes}
    </list>""".format(nodes=nodes, items_to_remove=items_to_remove)
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        temperature=0.1,
        max_tokens=4024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": base64_image,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ],
    )
    
    nodes = message.content[0].text.split("## Corrected list:")[-1].strip()
    return nodes
    
# nodes = correct_nodes(img_path, nodes)
# print(nodes)
# print(correct_nodes(img_path, nodes))

In [22]:
oneshot_assistant = """step_0: |
  - Determine user request intent
    - If user request is about personal (not related to a plumbing request), go to step_1
    - If user request is about selling something, go to step_2
    - If user request is about a new lead (needs plumbing services), go to step_3
    - If user request is about customer service (Past customer or customer with current booking), go to step_16

step_1: |
  - Tell the caller you will pass along a message and collect: Name Callback Number Message
  - Follow up with customer, if needed

step_2: |
  - Politely decline and insist you're not interested
  - End the call

step_3: |
  - Collect and log Basic Information in Talkdesk Contact: Name, Address, Phone Number, Email Address
  - Then go to step_4

step_4: |
  - Verify customer is in service range using the customer Map
    - If customer is NOT in service range, go to step_5
    - If customer is in service range, go to step_6

step_5: |
  - Log customer information in Talkdesk Contact
  - Let the customer know we are unable to service them
  - End the call

step_6: |
  - Refer to Knowledge Base to capture relevant information and verify we can service
    - If we cannot service, go to step_5
    - Else, go to step_7

step_7: |  
  - Confirm customer's distance from Portland, ME using the Service Zone Lookup tool
    - If customer is more than 60 minutes away, go to step_5
    - If customer is less than 60 minutes away and needs Estimate booking, go to step_8
    - If customer is less than 60 minutes away and needs Service Call booking, go to step_11

step_8: |
  - Determine how far the customer is
    - If less than 45 mins, go to step_9
    - If 45-60 mins, go to step_10

step_9: |
  - Disclose $90 dispatch fee (creditable to project cost)
  - Schedule next available estimate appointment
  - End the call

step_10: |
  - Disclose $139.50 dispatch fee
  - Schedule next available estimate appointment
  - End call

step_11: |
  - Determine if service is an emergency. Some example questions:"
        + Is your house currently flooding?
        + Can you shut the water off/do you know where the shutoffs are?
        + How long has this been going on for? 
      "
    - If yes, go to step_12
    - If no, go to step_15

step_12: |
  - Disclose $285 emergency dispatch fee and confirm customer is ready to proceed.
  - If customer is ready to proceed, go to step_27
  - If customer is not ready to proceed and wants to book regular appointment booking, go to step_15

step_27: |
  - Check booking availability for emergency
    - If after hours, go to step_13
    - If during business hours, go to step_14

step_13: |
  - Schedule appointment for first appointment the next morning
  - Call Andrew if true emergency (rare)
  - Commit to following up with customer

step_14: |
  - Schedule first available appointment
  - Contact Andrew to see if he can service the emergency sooner
  - Follow up with customer, if needed

step_15: |
  - Disclose $139 Dispatch Fee
  - Book customer for next available appointment in ServiceTitan
  - End call


step_16: |
  - Determine if customer is calling about UPCOMING booking or PAST/COMPLETED booking
    - If UPCOMING booking, go to step_17
    - If PAST/COMPLETED booking, go to step_24

step_17: |
  - Ask user to determine what the customer needs help with
    - If customer wants to change an upcoming appointment, go to step_18
    - If customer wants an updated arrival window, go to step_20
    - If customer wants to cancel upcoming appointment, go to step_23

step_18: |
  - Get upcoming appointment time
    - If upcoming appointment is today, go to step_19
    - If upcoming appointment is not today, go to step_15

step_19: |
  - Change the appointment and send an SMS message to the owner immediately to inform them of the same day scheduling changes
  - End call

step_20: |
  - Get quoted window information
    - If during/after quoted window, go to step_21
    - If before quoted window, go to step_22

step_21: |
  - Tell customer you will try to contact the plumber
  - Call Andrew asking about his arrival
  - Follow up with customer as needed

step_22: |
  - Inform the customer that their current arrival window is still the most accurate information. The plumber will contact when he is on the way
  - End call

step_23: |
  - Cancel the appointment
  - Ask customer why they are cancelling and record in call notes
  - If same day, send SMS to Andrew

step_24: |
  - Determine the reason for their call
    - It is urgent / customer is upset, go to step_25
    - If Customer has a non-urgent question about their service, go to step_26
    - If Customer wants to book another appointment, go to step_15

step_25: |
  - Record Name, callback number, issue and reach out to Andrew with the issue via SMS at any time of the day
  - Complete any required follow up with the customer

step_26: |
  - Record the customer's name, contact info, and question and put into the Summary on Service Titan
  - End call"""

In [23]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
anthropic_key = user_secrets.get_secret("ANTHROPIC_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = anthropic_key
# os.environ['OPENAI_API_KEY'] = "jjj"
import anthropic
import base64
from textwrap import dedent

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def build_diagram_text(nodes: list, img_path):
    print("Building diagram text...")
    client = anthropic.Anthropic()
    # oneshot_img = encode_image("/kaggle/working/imgs/Wild Pine Plumbing Service Call Tree-1.jpeg")
    oneshot_plan_img = encode_image("/kaggle/working/imgs/AT Plumbing Service Call Tree-1.jpeg")
    oneshot_hours_img = encode_image("/kaggle/input/plumbing/duringhours.jpeg")
    oneshot_hours_assistant = """  step_1: |
        - Determine if the service is an emergency
        - If you are unsure, call the on-call tech to determine if it is an emergency
        - If it is an emergency (needs service ASAP), go to step_2
        - If it is NOT an emergency, go to step_3
      step_2: |
        - Determine if the emergency is during or after business hours.
        - If it is during business hours, go to step_4
        - If it is after hours, go to step_5
      step_4: |
        - Tell the customer: We are not currently providing call answering during business hours, but would call Shane
      step_5: |
        - Disclose there are free estimates, but there is an increased price for after hours appointments
        - If the customer still wants emergency appointment, go to step_6
      step_6: |
        - Call the on- call tech (schedule found in KB) to confirm availability and timing, follow up with the customer as needed""" 
    main_image = encode_image(img_path)
    print("Main image:", img_path)
    
    oneshot_prompt = """The diagram in the provided image is a workflow process that describes steps or nodes that an customer service human agent handles phone calls from a person in different cases.
    Your task is to interpret the diagram and generate workflow plan in YAML format.
    Each box means a step.
    ### Strictly comply with the following rules:
    - In the output, include all boxes and texts found in the diagram, and keep the texts as they are. Expand texts are they are in the original diagram.
    - In the output, each Step corresponds with a box of any shape (rectangle, haxagon or diamond). Include all the nodes until you find no more.
    - For a current decision step that leads to multiple possible steps, use if-clauses in the current step to show steps that may conditionally come next. This is crucial for your success.
    - In the If-clause or in every Step, always use "go to step_<step_id>" as the reference to the next step. This is extremely important.
    - Carefully identify steps that follow the current step, especially steps that are connected to the current step with a long arrow line.
    These are extremely important requirements you must follow.
    """
    
    adhoc_rules = dedent("""
    - If there's "Angi Call", show it as a condition for a separate final step that follows Lead.
    """).strip()
    main_prompt = """Based on the example and rules above, interpret the latest diagram and generate workflow plan in YAML format.
    Consider the specific details of this diagram.
    The following elements are steps or conditions in the diagram. You have to consider ALL OF them together with the diagram when creating the output. Ensure to use all the elements in your output because they're the essential nodes and conditions.
    ### Elements to consider:
    <elements>
    {nodes}
    </elements>
    Go over each the elements and check you must include all the above elements in your output. Put them in the correct step where they belong to in the diagram.
    Ensure the output is a valid YAML format. Remember to use Literal Block Scalar Indicator.
    Additional rules:
    {adhoc_rules}
    """.format(nodes=nodes, adhoc_rules=adhoc_rules)
    #- If a step requires verification or confirmation it is followed by more than one step, you must write if-clauses for each case. This is extremely important because it ensures correct branching logic.
    
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        temperature=0.1,
        max_tokens=4024,
        messages=[
                {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": oneshot_hours_img,
                        },
                    },
                    {
                        "type": "text",
                        "text": oneshot_prompt
                    }
                ],
            },
            {
                "role": "assistant", 
                 "content": oneshot_hours_assistant
            },
            ### end of example 1
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": oneshot_plan_img,
                        },
                    },
                    {
                        "type": "text",
                        "text": oneshot_prompt
                    }
                ],
            },
            {
                "role": "assistant", 
                 "content": oneshot_assistant
            },
            #### end of example 2
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": main_image,
                        },
                    },
                    {
                        "type": "text",
                        "text": main_prompt
                    }
                ],
            }
        ],
    )
    
    diagram_text = message.content[0].text
    return diagram_text

# diagram_text = build_diagram_text(nodes, img_path)
# print(diagram_text)

In [30]:
reflect_prompt = dedent("""
    The following text is a way to describe the nodes and their relationships in the diagram from the provided image.
    <text>
    {diagram_text}
    </text>
    
    Here are a list of ELEMENTS that an ideal text for the diagram should include. The ELEMENTS include content of a step and conditions to go into that step.
    <elements>
    {elements}
    </elements>
    
    Your tasks are to:
    1. meticulously comparing the diagram text against each ELEMENT of the ELEMENTS list to see if the text misses any ELEMENTS. If it does, add those ELEMENTS to the text, using the diagram as a reference. Remember to include both step content and step conditions from the ELEMENTS LIST into your output.
    2. revise whether the diagram text correctly contains conditional logic. This means if a step requires verification or confirmation it is followed by more than one step, you must write if-clauses for different steps that follow the current step. This is extremely important because it ensures correct flow logic.
    3. revise the text meticulously to find any contents that are misplaced and should be placed in a different step according to the diagram. This is extremely important to correct workflow.
    4. Split steps that have nested if clauses into steps with just one-level if clauses.
    
    You should return:
    - Misplaced texts.
    - Steps with nested, multilevel if conditions. Reduce such steps into simple one-level if. When you add a new step to the output, use incremental integers to refer to the step ID. For example, use "step_2" rather than "step_2a". The new steps split from this must also have only one-level if clauses.
    - For each ELEMENT, show if the diagram text includes it or not. Respect orignal wordings. And be careful in distinguishing similar texts so as to put them in the correct step.
    - Steps that does not contain the correct logic of conditional if according to the given diagram.
    - Identify steps that involve verification or confirmation.
    - Then based on the above steps, identify steps that must be followed by another step according to the diagram but the text did not require it to be followed by any other steps. A step that is not terminal in the diagram must be followed by another step.
    - Find the step that "regular appointment booking" points to. Include this step in the diagram text and correctly reference it in the steps that uses it.
    - A detailed revised diagram text in valid YAML for all steps, based on the above results. Remember to use Literal Block Scalar Indicator. The new text must be complete, showing all elements and contain no notes so that when you read it, there is no `[...steps ... remain the same]` and `[Continued ... due to length...]` and you won't know that modifications are there. Add title "### Revised text:" before the text. Ignore output length limits if the final text is lengthy. Just show all in this turn. This is crucially important.
    
    """).strip()
def revise_diagram_text(diagram_text: str, nodes: list, img_path):
    print("Revising diagram text...")
    inside_reflect_prompt = reflect_prompt.format(diagram_text=diagram_text, elements=nodes)
    client = anthropic.Anthropic()
    main_image = encode_image(img_path)
    reflect_message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        temperature=0.4,
        max_tokens=8024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": main_image,
                        },
                    },
                    {
                        "type": "text",
                        "text": inside_reflect_prompt
                    }
                ],
            }
        ],
    )
    
    reflect = reflect_message.content[0].text
    return reflect

# reflect_result = revise_diagram_text(diagram_text, nodes, img_path)
# print(reflect_result)

In [31]:
def expand_answer(diagram_text, imag_path: str, reflect_result: str) -> str:
    client = anthropic.Anthropic()
    main_image = encode_image(img_path)
    filled_reflect_prompt = reflect_prompt.format(diagram_text=diagram_text, elements=nodes)
    expansion_prompt = "Yes, please. Obey the rules above. Use integer to number the steps. Ensure you give valid yaml format. And don't add any notes."
    reflect_message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    temperature=0.4,
    max_tokens=8024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": main_image,
                    },
                },
                {
                    "type": "text",
                    "text": filled_reflect_prompt
                }
            ],
        },
        {
        "role": "assistant", 
         "content": reflect_result
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": expansion_prompt
                }
            ],
        },
    ],
    )

    full_show = reflect_message.content[0].text
    full_show = extract_result(full_show)
    return full_show.strip()

def fix_yaml(yaml_text) -> str:
    client = anthropic.Anthropic()
    fixing_prompt = dedent("""The following text has invalid YAML format. Correct it and give a valid yaml. Remember to use Literal Block Scalar Indicator. Don't add any notes.
    <text>
    {yaml_text}
    </text>
    """).strip().format(yaml_text=yaml_text)
    fixing_message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        temperature=0.4,
        max_tokens=8024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": fixing_prompt
                    }
                ],
            },
        ],
        )

    return fixing_message.content[0].text.strip()

In [33]:
import yaml
import chime
from tenacity import retry, stop_after_attempt

def extract_result(text: str) -> str:
    if "### Revised text" in text:
        revised_text = (text.split("### Revised text:", 1)[1]
                        .replace("```yaml", "")
                        .replace("```", "")
                        .strip()
                       )
    else:
        revised_text = None 
    return revised_text

def validate_yaml(yaml_text):
    try:
        yaml.safe_load(yaml_text)
        return "true"
    except yaml.YAMLError as e:
        return "invalid"


def textualize(nodes):
    print("Starting...")
    nodes = correct_nodes(img_path, nodes)
    diagram_text = build_diagram_text(nodes, img_path)
    reflect_result = revise_diagram_text(diagram_text, nodes, img_path)
    print("@@@", reflect_result)
    yaml_text = extract_result(reflect_result)
    val_result = validate_yaml(yaml_text)
    if val_result == "true":
        print("First run OK")
        return "OK"
    elif val_result == "invalid":
        print("Need full text")
        full_show = expand_answer(diagram_text, img_path, reflect_result)
        check = validate_yaml(full_show)
        if check == "true":
            print("Yaml OK")
            return full_show.strip()
        else:
            print("Invalid yaml. Fixing it...")
            return fix_yaml(full_show)

print(textualize(nodes))
chime.success()

Starting...
Correcting nodes...
Building diagram text...
Main image: /kaggle/working/imgs/AT Plumbing Service Call Tree-1.jpeg
Revising diagram text...
@@@ I'll analyze each aspect carefully:

1. ELEMENTS Check:
Missing elements:
- "Book next available appointment" (Element from step_12) is not properly connected
- "Regular appointment booking" (Element 50) is missing its destination step
- Element 54 "Schedule" is not properly included
- Element 58 "Disclose $139 Dispatch Fee" needs to be more explicitly stated

2. Steps with nested, multilevel if conditions that need splitting:
- step_7 has multiple nested conditions about appointment changes and windows
- step_19 has nested conditions about estimate/service call booking
- step_21 has nested conditions about emergency timing
- step_23 has nested conditions about business hours

3. Misplaced texts:
- The emergency dispatch fee disclosure should come before the scheduling options in step_23
- The verification of service range should co

## EVALUATION
### Criteria
- Include all steps
- Include all conditions for steps
- Maintain original step orders
- Use correct step branching
- Include actionable statement at beginning of step
- Valid YAML